In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import folium.plugins
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
daniela_path = 'drive/MyDrive/Class/CS209b/CS209b_Final_Project'
elie_path = "drive/MyDrive/CS209b_Final_Project"

path = elie_path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Web Scraping

To create our dataset, we used to following code to scrape the ZIP code and screenshot from the location generated on randomstreetview.com

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

#Create list to store zip codes
zip_list = []

#Initialize driver
url = 'https://randomstreetview.com/us'
DRIVER = '~/Users/lu31635/chromedriver'
driver = webdriver.Chrome(DRIVER)
num_images = 1000
starting_num = 9000

#generate 10 random locations
for i in range(num_images):
    #get random street view webpage
    driver.get(url)
    time.sleep(3)
    #grab screenshot of page
    screenshot = driver.save_screenshot(f'project/images/location{i+starting_num}.png')
    #scrape address
    address = driver.find_element(By.ID, 'address').text
    #isolate zipcode and save to dictionary
    zipcode = address.split(',')[-2][-5:]
    zip_list.append([i+starting_num, zipcode])

driver.quit()

#write zip codes to csv
with open('project/zipcodes8.csv', 'w') as outfile:
    writer = csv.writer(outfile)
    for line in zip_list:
        writer.writerow(line)


## Cropping and resizing images

In [ ]:
#This cell was used to crop and resize the scraped images
from PIL import Image

for i in range(10000):
    im = Image.open(f'project/images/location{i}.jpg')
    #crop image to remove site UI
    cropped = im.crop((500, 80, 2300, 1300))
    #resize to 250x170 for CNN
    resized = cropped.resize((250,170))
    path = f'project/croppedimages/location{i}.jpg'
    #save new image
    resized.save(path)

In [ ]:
from PIL import Image

counter = 0
for i in range(1,1100):
    for j in range(6):
        if i < 10:
            im = Image.open(f'project/part1/00000{i}_{j}.jpg')
        elif i < 100:
            im = Image.open(f'project/part1/0000{i}_{j}.jpg')
        elif i < 1000:
            im = Image.open(f'project/part1/000{i}_{j}.jpg')
        else:
            im = Image.open(f'project/part1/00{i}_{j}.jpg')

        if j in [1,2,3,4]:
            #crop image to remove site UI
            cropped = im.crop((0, 80, 1280, 950))
            #resize to 250x170 for CNN
            resized = cropped.resize((250,170))
            path = f'project/croppedimages/location{counter+10000}.jpg'
            #save new image
            resized.save(path)
            counter += 1

## Creating Outcome Variables

In [ ]:
y = pd.read_csv(path + '/outcome_data/election2022_vF.csv')#[['District', 'DEM', 'GEOID']]

In [ ]:
metadata = pd.read_csv(path + '/location_data/final_locations.csv')
metadata.head()

,ID,ZIP,Latitude,Longitude,City,State,County,Zip,Country,Congressional District
0,0,4401,44.848517,-68.850405,Bangor,ME,Penobscot County,4401.0,US,ME2
1,1,68050,41.314461,-96.823281,Prague,NE,Saunders County,68050.0,US,NE2
2,2,58579,47.158600,-100.783470,Wilton,ND,McLean County,58579.0,US,ND0
3,3,99402,46.339330,-117.048210,Asotin,WA,Asotin County,99402.0,US,WA5
4,4,32352,30.657410,-84.691030,Mount Pleasant,FL,Gadsden County,32352.0,US,FL2


In [ ]:
metadata = metadata.drop(metadata[metadata['ZIP'] == 99999].index)
metadata['Congressional District'] = metadata['Congressional District'].apply(lambda x: x[:2] + "-" + x[2:])

In [ ]:
metadata['Congressional District']

0       ME-2
1       NE-2
2       ND-0
3       WA-5
4       FL-2
        ... 
9995    WI-8
9996    WI-1
9997    MO-4
9998    CO-2
9999    ID-2
Name: Congressional District, Length: 9996, dtype: object

In [ ]:
#Limit dataset to necessary variables and merge with midterm results data
y_full = metadata[['ID', 'Congressional District', 'Latitude', 'Longitude']].merge(y[['District', 'DEM']], left_on="Congressional District", right_on="District", how='right')

In [ ]:
y_full = y_full[['ID', 'DEM', 'Latitude', 'Longitude']]

In [ ]:
#Add election results for added urban locations (all democrat)
for i in range(10000,14000):
    new_row = {'ID': int(i), 'DEM': 1, 'Latitude': 0, 'Longitude': 0}
    y_full = y_full.append(new_row, ignore_index = True)

In [ ]:
#display updated dataframe
y_full

,ID,DEM,Latitude,Longitude
0,338.0,0,31.129427,-87.096126
1,347.0,0,31.090730,-88.228060
2,960.0,0,30.666959,-88.226989
3,1268.0,0,30.509877,-88.173001
4,1767.0,0,30.509877,-88.173001
...,...,...,...,...
13653,13995.0,1,0.000000,0.000000
13654,13996.0,1,0.000000,0.000000
13655,13997.0,1,0.000000,0.000000
13656,13998.0,1,0.000000,0.000000


In [ ]:
#Updated Republican and Demorat district counts
value_counts = y_full['DEM'].value_counts()
value_counts

0    6927
1    6731
Name: DEM, dtype: int64

In [ ]:
#save dataframe to csv file
y_full.to_csv(path + "/outcome_data/final_outcome.csv")